In [3]:
# Basic Libraries
import numpy as np
import pandas as pd

In [9]:
# Load the CSV dataset into a Pandas dataframe
rawCOE = pd.read_csv('Results of COE Bidding Exercise - Results.csv')
rawCOE.head()

,Bidding Exercise,Announcement Date,Year,Category,Quota,Quota Premium,Total Bids Received,Number of Successful Bids
0,March 2023 First Open Bidding Exercise,08/03/2023,2023,Cat A (Cars up to 1600cc and 97kW),517,"$88,000",665,507
1,March 2023 First Open Bidding Exercise,08/03/2023,2023,Cat B (Cars above 1600cc or 97kW),431,"$115,501",629,431
2,March 2023 First Open Bidding Exercise,08/03/2023,2023,Cat C (Goods vehicles and buses),43,"$91,101",65,39
3,March 2023 First Open Bidding Exercise,08/03/2023,2023,Cat D (Motorcycles),467,"$12,390",566,465
4,March 2023 First Open Bidding Exercise,08/03/2023,2023,Cat E (Open),132,"$116,000",254,128


In [11]:
# Extract the month and year from the "Bidding Exercise" column and store it in a new "Month Year" column
relevantData = rawCOE.copy()
relevantData['Month Year'] = relevantData['Bidding Exercise'].apply(lambda x: pd.to_datetime(x.split()[0] + " " + x.split()[1]).strftime('%b %Y'))
relevantData['Month Year'].head()

0    Mar 2023
1    Mar 2023
2    Mar 2023
3    Mar 2023
4    Mar 2023
Name: Month Year, dtype: object

In [15]:
# Create a new dataframe that contains the transformed "Month Year", "Category", and "Quota Premium" columns
relevantData = relevantData[['Month Year', 'Category', 'Quota Premium']]

# Exclude rows that contain "Cat C", "Cat D", or "Cat E" in the "Category" column
relevantData = relevantData[~relevantData['Category'].str.contains('Cat C|Cat D|Cat E')]

relevantData

,Month Year,Category,Quota Premium
0,Mar 2023,Cat A (Cars up to 1600cc and 97kW),"$88,000"
1,Mar 2023,Cat B (Cars above 1600cc or 97kW),"$115,501"
5,Feb 2023,Cat A (Cars up to 1600cc and 97kW),"$86,556"
6,Feb 2023,Cat B (Cars above 1600cc or 97kW),"$115,001"
10,Feb 2023,Cat A (Cars up to 1600cc and 97kW),"$86,000"
...,...,...,...
2471,Nov 2002,Cat B (Cars above 1600cc or 97kW),"$28,001"
2475,Dec 2002,Cat A (Cars up to 1600cc and 97kW),"$28,501"
2476,Dec 2002,Cat B (Cars above 1600cc or 97kW),"$27,289"
2480,Dec 2002,Cat A (Cars up to 1600cc and 97kW),"$26,993"


In [19]:
relevantData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 0 to 2481
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month Year     994 non-null    object
 1   Category       994 non-null    object
 2   Quota Premium  994 non-null    object
dtypes: object(3)
memory usage: 31.1+ KB


In [21]:
# Remove dollar sign and comma characters from "Quota Premium" column
relevantData['Quota Premium'] = relevantData['Quota Premium'].replace({'\$': '', ',': ''}, regex=True)

# Convert the "Quota Premium" column to int64
relevantData['Quota Premium'] = relevantData['Quota Premium'].astype('int64')

relevantData

,Month Year,Category,Quota Premium
0,Mar 2023,Cat A (Cars up to 1600cc and 97kW),88000
1,Mar 2023,Cat B (Cars above 1600cc or 97kW),115501
5,Feb 2023,Cat A (Cars up to 1600cc and 97kW),86556
6,Feb 2023,Cat B (Cars above 1600cc or 97kW),115001
10,Feb 2023,Cat A (Cars up to 1600cc and 97kW),86000
...,...,...,...
2471,Nov 2002,Cat B (Cars above 1600cc or 97kW),28001
2475,Dec 2002,Cat A (Cars up to 1600cc and 97kW),28501
2476,Dec 2002,Cat B (Cars above 1600cc or 97kW),27289
2480,Dec 2002,Cat A (Cars up to 1600cc and 97kW),26993


In [23]:
# Group the dataframe by "Month Year" and calculate the average of the "Quota Premium" values
relevantData = relevantData.groupby('Month Year')['Quota Premium'].mean().reset_index()

# Rename the "Quota Premium" column to "Average Premium"
relevantData = relevantData.rename(columns={'Quota Premium': 'Average Premium'})

relevantData

,Month Year,Average Premium
0,Apr 2002,38702.75
1,Apr 2003,27302.50
2,Apr 2004,27569.00
3,Apr 2005,17547.75
4,Apr 2006,12576.25
...,...,...
244,Sep 2018,30379.25
245,Sep 2019,34135.75
246,Sep 2020,39692.25
247,Sep 2021,56477.50


In [30]:
# Convert the "Month Year" column to datetime format
relevantData['Month Year'] = pd.to_datetime(relevantData['Month Year'], format='%b %Y')
# Sort the rows by date
relevantData = relevantData.sort_values('Month Year')
# Reset the index of the rows
relevantData = relevantData.reset_index(drop=True)
# Format the "Month Year" column as "MMM YYYY"
relevantData['Month Year'] = relevantData['Month Year'].dt.strftime('%b %Y')
relevantData

,Month Year,Average Premium
0,Apr 2002,38702.75
1,May 2002,36600.75
2,Jun 2002,32197.50
3,Jul 2002,30948.75
4,Aug 2002,31367.00
...,...,...
244,Nov 2022,101773.25
245,Dec 2022,97504.25
246,Jan 2023,93365.25
247,Feb 2023,98270.25


In [31]:
relevantData.to_csv('average COE prices per month.csv', index=False)